### This project is to load the bitcoin CSV data from data.world website and transform the data to required data types and to load into the DB table

In [1]:
import pandas as pd
from sqlalchemy import create_engine

#### Store CSV into DataFrame

In [2]:
csv_file = "../Resources/bitcoin_data.csv"
bitcoin_data_df = pd.read_csv(csv_file)
bitcoin_data_df.head()

,Timestamp,Open,High,Low,Close,Volume (BTC),Volume (Currency),Weighted Price
0,1/1/17 0:00,966.34,1005.00,960.53,997.75,6850.59,6764742.06,987.47
1,1/2/17 0:00,997.75,1032.00,990.01,1012.54,8167.38,8273576.99,1013.00
2,1/3/17 0:00,1011.44,1039.00,999.99,1035.24,9089.66,9276500.31,1020.56
3,1/4/17 0:00,1035.51,1139.89,1028.56,1114.92,21562.46,23469644.96,1088.45
4,1/5/17 0:00,1114.38,1136.72,885.41,1004.74,36018.86,36211399.53,1005.35


In [3]:
#Change column names

In [4]:
bitcoin_data_df = bitcoin_data_df.rename(columns = {"Timestamp":"trans_date",
                                                    "Close":"price",
                                                    "Open":"openvalue",
                                                    "High":"dayhighvalue",
                                                    "Low":"daylowvalue",
                                                    "Volume (BTC)":"volume_btc",
                                                    "Volume (Currency)":"volume_currency",
                                                    "Weighted Price":"weighted_price"} )
bitcoin_data_df.drop_duplicates("trans_date", inplace =True)
bitcoin_data_df.head()

,trans_date,openvalue,dayhighvalue,daylowvalue,price,volume_btc,volume_currency,weighted_price
0,1/1/17 0:00,966.34,1005.00,960.53,997.75,6850.59,6764742.06,987.47
1,1/2/17 0:00,997.75,1032.00,990.01,1012.54,8167.38,8273576.99,1013.00
2,1/3/17 0:00,1011.44,1039.00,999.99,1035.24,9089.66,9276500.31,1020.56
3,1/4/17 0:00,1035.51,1139.89,1028.56,1114.92,21562.46,23469644.96,1088.45
4,1/5/17 0:00,1114.38,1136.72,885.41,1004.74,36018.86,36211399.53,1005.35


In [5]:
rds_connection_string = "postgres:admin@localhost:5432/finance_db"
#<insert password>@localhost:5432/customer_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [6]:
engine.table_names()

['sandp',
 'btc_usd',
 'yahoo_most_active',
 'yahoo_most_trending',
 'yahoo_btc_coin']

In [18]:
bitcoin_data_df.to_sql(name='btc_usd', con=engine, if_exists='append', index=False)

In [19]:
pd.read_sql_query('select * from btc_usd', con=engine).head()

,trans_date,price,openvalue,dayhighvalue,daylowvalue,volume_btc,volume_currency,weighted_price
0,2017-01-01,997.75,966.34,1005.00,960.53,6851,6764742,987.47
1,2017-01-02,1012.54,997.75,1032.00,990.01,8167,8273577,1013.00
2,2017-01-03,1035.24,1011.44,1039.00,999.99,9090,9276500,1020.56
3,2017-01-04,1114.92,1035.51,1139.89,1028.56,21562,23469645,1088.45
4,2017-01-05,1004.74,1114.38,1136.72,885.41,36019,36211400,1005.35
